In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["LANGCHAIN_API_KEY"]=os.environ.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]="tavily"

In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults

tool = TavilySearchResults(
    max_results=5,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    include_images=True,
    # include_domains=[...],
    # exclude_domains=[...],
    # name="...",            # overwrite default tool name
    # description="...",     # overwrite default tool description
    # args_schema=...,       # overwrite default args_schema: BaseModel
)

# Direct tool invocation

In [3]:
tool.invoke({"query": "What happened at the last wimbledon"})

[{'url': 'https://www.nbcnews.com/news/sports/andy-murray-wimbledon-tennis-singles-draw-rcna159912',
  'content': "By Patrick Smith. LONDON — Andy Murray, one of the last decade's most successful male tennis players, has pulled out of the singles tournament at what is almost certain to be his last Wimbledon ..."},
 {'url': 'https://www.theguardian.com/sport/live/2023/jul/16/wimbledon-mens-singles-final-2023-carlos-alcaraz-v-novak-djokovic-live?page=with:block-64b3ff568f08df28470056bf',
  'content': 'Carlos Alcaraz recovered from a set down to topple Djokovic 1-6, 7-6(6), 6-1, 3-6, 6-4 and win his first Wimbledon title in a battle for the ages'},
 {'url': 'https://www.cnn.com/2024/07/09/sport/novak-djokovic-wimbledon-crowd-quarterfinals-spt-intl/index.html',
  'content': 'Novak Djokovic produced another impressive performance at Wimbledon on Monday to cruise into the quarterfinals, but the 24-time grand slam champion was far from happy after his win. The Serb took ...'},
 {'url': 'https

# Invokation via tools call

In [4]:
# This is usually generated by a model, but we'll create a tool call directly for demon purposes.
model_generated_tool_call = {
    "args": {"query": "euro 2024 host nation"},
    "id": "1",
    "name": "tavily",
    "type": "tool_call",
}
tool_msg = tool.invoke(model_generated_tool_call["args"])

# The content is a JSON string of results
print(tool_msg)

[{'url': 'https://www.sportingnews.com/ca/soccer/news/list-euros-host-nations-uefa-european-championship-countries/85f8069d69c9f4ecd00c4900', 'content': 'The 2024 UEFA European Championship, more commonly known as Euro 2024, will mark the 17th edition of the tournament and is set to be hosted by Germany from June 14 to July 14.'}, {'url': 'https://footballgroundguide.com/euro-2024-host-nation', 'content': 'As the Euro 2024 host nation, Germany will play the opening match against Scotland on 14 June before clashing with Hungary and finally Switzerland in the group stages. It is not an easy group but Germany is the highest-ranked by FIFA in group A of the Euro 2024. Apart from European Championship access, Germany has also been a powerhouse in ...'}, {'url': 'https://www.sportingnews.com/us/soccer/news/where-euro-2024-host-nation-next-european-championships/fguxslt9krlghuf1rcn9yf4v', 'content': "Germany will host Euro 2024, the next men's European Championships, from June 14 to July 14, 

In [5]:
content_list = [item['content'] for item in tool_msg]

# Print the extracted content
for content in content_list:
    print(content)


The 2024 UEFA European Championship, more commonly known as Euro 2024, will mark the 17th edition of the tournament and is set to be hosted by Germany from June 14 to July 14.
As the Euro 2024 host nation, Germany will play the opening match against Scotland on 14 June before clashing with Hungary and finally Switzerland in the group stages. It is not an easy group but Germany is the highest-ranked by FIFA in group A of the Euro 2024. Apart from European Championship access, Germany has also been a powerhouse in ...
Germany will host Euro 2024, the next men's European Championships, from June 14 to July 14, 2024. Find out the 10 stadiums, the schedule, the format and the bidding process for the tournament.
UEFA EURO 2024. Host cities. Berlin Cologne Munich Frankfurt Hamburg Dortmund Leipzig Gelsenkirchen Stuttgart Düsseldorf. UEFA EURO 2024. Matches. Groups. Video. Stats. Gaming.
Find out where and when Euro 2024 will take place, and which cities and stadiums will host the matches. Ger

# Chaining

We can use our tool in a chain by first binding it to a tool-calling model and then calling it:



In [6]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")


In [30]:
import datetime

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableConfig, chain

today = datetime.datetime.today().strftime("%D")
today

'08/18/24'

In [32]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage

prompt = ChatPromptTemplate.from_messages([
    ("system", f"You are a helpful assistant. The date today is {today}."),
    ("user", "{user_input}"),
    ("placeholder", "{messages}"),
])

In [33]:

# specifying tool_choice will force the model to call this tool.
llm_with_tools = llm.bind_tools([tool])

llm_chain = prompt | llm_with_tools


@chain
def tool_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    tool_msgs = tool.batch(ai_msg.tool_calls, config=config)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)



In [36]:
query = "who won the last womens singles wimbledon"

In [41]:
tool_chain.ainvoke({"input": query})

<coroutine object RunnableLambda.ainvoke at 0x0000023D59780270>

In [42]:
output=tool_chain.ainvoke("who won the last womens singles wimbledon")

In [43]:
output

<coroutine object RunnableLambda.ainvoke at 0x0000023D59780740>